In [1]:
import pandas as pd
import numpy as np
from taigapy import create_taiga_client_v3
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
Model = pd.read_pickle("pydata/model_data.pkl")

In [ ]:
Model

,ModelID,PatientID,CellLineName,StrippedCellLineName,DepmapModelType,OncotreeLineage,OncotreePrimaryDisease,OncotreeSubtype,OncotreeCode,PatientSubtypeFeatures,...,EngineeredModelDetails,CulturedResistanceDrug,PublicComments,CCLEName,HCMIID,ModelAvailableInDbgap,ModelSubtypeFeatures,WTSIMasterCellID,SangerModelID,COSMICID
0,ACH-000001,PT-gj46wT,NIH:OVCAR-3,NIHOVCAR3,HGSOC,Ovary/Fallopian Tube,Ovarian Epithelial Tumor,High-Grade Serous Ovarian Cancer,HGSOC,None,...,None,None,None,NIHOVCAR3_OVARY,None,Available in SRA,None,2201.0,SIDM00105,905933.0
1,ACH-000002,PT-5qa3uk,HL-60,HL60,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,"TP53(del), CDKN2A and NRAS mutations [PubMed=2...",...,None,None,None,HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,None,Available in SRA,None,55.0,SIDM00829,905938.0
2,ACH-000003,PT-puKIyc,CACO2,CACO2,COAD,Bowel,Colorectal Adenocarcinoma,Colon Adenocarcinoma,COAD,None,...,None,None,None,CACO2_LARGE_INTESTINE,None,Available in SRA,None,NaN,SIDM00891,NaN
3,ACH-000004,PT-q4K2cp,HEL,HEL,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,"JAK2 and TP53 mutations,",...,None,None,None,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,None,Available in SRA,None,783.0,SIDM00594,907053.0
4,ACH-000005,PT-q4K2cp,HEL 92.1.7,HEL9217,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,JAK2 and TP53 mutations,...,None,None,None,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,None,Available in SRA,None,NaN,SIDM00593,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2547,ACH-003659,PT-UFmFPI,SMS-KAN,SMSKAN,NBL,Peripheral Nervous System,Neuroblastoma,Neuroblastoma,NBL,None,...,None,None,None,None,None,None,None,NaN,None,NaN
2548,ACH-003660,PT-UFmFPI,SMS-KANR,SMSKANR,NBL,Peripheral Nervous System,Neuroblastoma,Neuroblastoma,NBL,None,...,None,None,None,None,None,None,None,NaN,None,NaN
2549,ACH-003665,PT-ZwaNYv,MBU-8,MBU8,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,"Compared to the parent cell line, has a comple...",...,None,None,None,None,None,None,None,NaN,None,NaN
2550,ACH-003666,PT-ijqCP4,YCUB-4,YCUB4,BLL,Lymphoid,B-Lymphoblastic Leukemia/Lymphoma,B-Lymphoblastic Leukemia/Lymphoma,BLL,cells express a MEF2D exon5/BCL9 exon 9 fusion,...,None,None,None,None,None,None,MEF2D,NaN,None,NaN


In [4]:
Model.set_index('ModelID', inplace=True)

In [ ]:
Model

,PatientID,CellLineName,StrippedCellLineName,DepmapModelType,OncotreeLineage,OncotreePrimaryDisease,OncotreeSubtype,OncotreeCode,PatientSubtypeFeatures,RRID,...,EngineeredModelDetails,CulturedResistanceDrug,PublicComments,CCLEName,HCMIID,ModelAvailableInDbgap,ModelSubtypeFeatures,WTSIMasterCellID,SangerModelID,COSMICID
ModelID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,PT-gj46wT,NIH:OVCAR-3,NIHOVCAR3,HGSOC,Ovary/Fallopian Tube,Ovarian Epithelial Tumor,High-Grade Serous Ovarian Cancer,HGSOC,None,CVCL_0465,...,None,None,None,NIHOVCAR3_OVARY,None,Available in SRA,None,2201.0,SIDM00105,905933.0
ACH-000002,PT-5qa3uk,HL-60,HL60,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,"TP53(del), CDKN2A and NRAS mutations [PubMed=2...",CVCL_0002,...,None,None,None,HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,None,Available in SRA,None,55.0,SIDM00829,905938.0
ACH-000003,PT-puKIyc,CACO2,CACO2,COAD,Bowel,Colorectal Adenocarcinoma,Colon Adenocarcinoma,COAD,None,CVCL_0025,...,None,None,None,CACO2_LARGE_INTESTINE,None,Available in SRA,None,NaN,SIDM00891,NaN
ACH-000004,PT-q4K2cp,HEL,HEL,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,"JAK2 and TP53 mutations,",CVCL_0001,...,None,None,None,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,None,Available in SRA,None,783.0,SIDM00594,907053.0
ACH-000005,PT-q4K2cp,HEL 92.1.7,HEL9217,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,JAK2 and TP53 mutations,CVCL_2481,...,None,None,None,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,None,Available in SRA,None,NaN,SIDM00593,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-003659,PT-UFmFPI,SMS-KAN,SMSKAN,NBL,Peripheral Nervous System,Neuroblastoma,Neuroblastoma,NBL,None,CVCL_7131,...,None,None,None,None,None,None,None,NaN,None,NaN
ACH-003660,PT-UFmFPI,SMS-KANR,SMSKANR,NBL,Peripheral Nervous System,Neuroblastoma,Neuroblastoma,NBL,None,CVCL_7132,...,None,None,None,None,None,None,None,NaN,None,NaN
ACH-003665,PT-ZwaNYv,MBU-8,MBU8,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,"Compared to the parent cell line, has a comple...",CVCL_B3XV,...,None,None,None,None,None,None,None,NaN,None,NaN


In [7]:
E3_expr_filtered = pd.read_pickle("pydata/e3_expression_filtered.pkl")

In [44]:
E3_expr_filtered

,KDM2A,RNF145,RNF157,RNF165,RNF212,RNF39,SH3RF2,TRAF4,TRIM52,TRIM69,...,TRIM58,RNF183,DET1,FBXL18,FBXL22,FBXO21,FBXO34,FBXW12,SPSB3,NEURL1
ModelID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,5.350041,5.781840,0.546522,0.705279,1.197362,1.059804,2.521187,7.234615,3.183593,3.869171,...,0.164298,0.364297,2.134941,2.066240,0.478557,3.804434,4.763590,0.553823,5.609612,0.888082
ACH-000004,4.284098,5.500041,0.153294,-0.050736,0.006805,1.603985,-0.118006,5.283476,2.520298,3.332695,...,3.922365,0.004578,2.813761,1.068238,0.009448,1.024548,4.014569,0.105832,3.671588,0.032455
ACH-000005,4.778394,5.916353,0.153294,-0.036715,-0.033062,0.759238,-0.031406,5.271112,2.280233,3.506815,...,4.188531,0.004578,3.246952,1.199344,0.060415,2.145190,4.646387,0.146463,2.282317,0.226062
ACH-000007,4.224493,5.444087,0.973254,0.453291,0.155439,0.502655,4.343146,5.804416,3.777697,2.995643,...,-0.003895,0.078077,1.561329,1.867515,0.533023,3.833594,4.367864,0.116126,3.775234,1.592114
ACH-000009,5.983751,4.399241,3.985872,-0.009083,-0.006353,0.177682,5.073699,6.078089,2.672089,1.827145,...,0.009799,0.710466,0.844292,2.467066,0.035165,4.262128,3.748417,0.185705,3.350565,0.216067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-003578,4.968736,6.247850,3.229085,1.252558,0.028522,0.302531,1.243030,5.249282,1.927991,3.833978,...,0.118096,-0.012212,2.709784,2.210355,0.410324,4.479383,3.792392,0.188573,1.837676,0.465824
ACH-003580,3.701024,4.573275,5.132026,2.788532,0.028522,0.302531,0.145815,5.441013,0.823754,2.292339,...,0.002095,-0.012212,1.953351,0.819196,0.077013,4.154070,2.301319,1.375910,4.141284,1.761585
ACH-003587,4.601888,5.682004,3.519492,0.171958,0.028522,0.449147,2.889733,3.853863,2.846582,3.332735,...,3.527476,-0.012212,3.126775,2.770160,0.232896,4.668813,3.661855,0.014191,3.775213,0.762195


In [66]:
crispr_dependencies_3_filtered = pd.read_pickle("pydata/tfs_dependencies_filtered.pkl")

In [67]:
crispr_dependencies_3_filtered

,TFAP2A,TFAP2B,TFAP2C,TFAP2D,TFAP2E,ARID3A,ARID3B,ARID3C,ARID5A,ARID5B,...,THAP6,THAP7,THAP8,THAP9,BMAL1,BMAL2,ZNF875,CBLL2,ZUP1,TBXT
ModelID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,0.032275,-0.052657,-0.099978,-0.040024,0.060890,0.102625,-0.121782,0.027268,-0.243864,-0.096952,...,-0.087478,-0.317241,-0.052855,-0.065466,-0.645930,0.140806,-0.892911,0.215270,-0.014496,0.169385
ACH-000004,0.145169,0.066916,-0.074218,-0.125423,0.240744,-0.105335,-0.734358,-0.027738,-0.136107,0.008301,...,-0.462663,-0.194145,0.130992,-0.251477,-0.205806,-0.090015,0.168027,0.280951,-0.182743,0.082328
ACH-000005,0.171513,-0.361703,-0.017947,-0.103094,0.315080,0.069369,-0.026263,-0.022071,0.006219,0.200501,...,-0.094047,-0.180209,-0.163869,-0.026951,-0.207084,-0.134599,0.374316,-0.018200,-0.283811,0.058674
ACH-000007,-0.073170,0.200249,-0.292673,-0.238197,-0.059635,0.041934,-0.329979,-0.080568,-0.067746,-0.198359,...,-0.071877,-0.105574,0.144103,-0.023396,0.140064,-0.069550,0.180594,0.104287,-0.311871,-0.006005
ACH-000009,0.224490,-0.001485,-0.172446,0.018604,0.034670,-0.316335,-0.053869,0.059077,-0.058235,-0.495861,...,-0.059427,-0.024232,-0.025614,-0.115766,-0.111904,0.027901,0.098494,0.060150,0.014514,0.136429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-003578,-0.070363,-0.033106,-0.243238,-0.370448,-0.108596,0.023995,-0.181551,-0.213355,-0.284968,-0.052868,...,0.040754,-0.215718,0.054442,-0.043687,-0.052162,-0.106350,0.018423,0.079413,0.005381,0.105370
ACH-003580,-0.081792,-0.181257,-0.381345,-0.202155,0.109083,-0.004828,0.018773,-0.112690,-0.071700,-0.386106,...,-0.040535,-0.178553,0.195923,0.063442,-0.119621,-0.077457,-0.205756,-0.021425,0.016575,0.077086
ACH-003587,-0.128495,-0.347200,-0.229025,-0.420290,0.106401,-0.087998,0.062317,-0.158840,0.009397,-0.086930,...,0.022615,0.289244,0.065706,0.043732,-0.095881,0.210714,-0.104913,-0.141233,0.058059,0.013758


In [ ]:
Model_filtered = Model[Model.index.isin(E3_expr_filtered.index)]

In [ ]:
Model_filtered

,PatientID,CellLineName,StrippedCellLineName,DepmapModelType,OncotreeLineage,OncotreePrimaryDisease,OncotreeSubtype,OncotreeCode,PatientSubtypeFeatures,RRID,...,EngineeredModelDetails,CulturedResistanceDrug,PublicComments,CCLEName,HCMIID,ModelAvailableInDbgap,ModelSubtypeFeatures,WTSIMasterCellID,SangerModelID,COSMICID
ModelID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,PT-gj46wT,NIH:OVCAR-3,NIHOVCAR3,HGSOC,Ovary/Fallopian Tube,Ovarian Epithelial Tumor,High-Grade Serous Ovarian Cancer,HGSOC,None,CVCL_0465,...,None,None,None,NIHOVCAR3_OVARY,None,Available in SRA,None,2201.0,SIDM00105,905933.0
ACH-000004,PT-q4K2cp,HEL,HEL,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,"JAK2 and TP53 mutations,",CVCL_0001,...,None,None,None,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,None,Available in SRA,None,783.0,SIDM00594,907053.0
ACH-000005,PT-q4K2cp,HEL 92.1.7,HEL9217,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,JAK2 and TP53 mutations,CVCL_2481,...,None,None,None,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,None,Available in SRA,None,NaN,SIDM00593,NaN
ACH-000007,PT-NOXwpH,LS513,LS513,COAD,Bowel,Colorectal Adenocarcinoma,Colon Adenocarcinoma,COAD,None,CVCL_1386,...,None,None,None,LS513_LARGE_INTESTINE,None,Available in SRA,None,569.0,SIDM00677,907795.0
ACH-000009,PT-puKIyc,C2BBe1,C2BBE1,COAD,Bowel,Colorectal Adenocarcinoma,Colon Adenocarcinoma,COAD,None,CVCL_1096,...,None,None,None,C2BBE1_LARGE_INTESTINE,None,Available in SRA,None,2104.0,SIDM01233,910700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-003578,PT-XwatVo,ES8_SLFN11-KO,ES8SLFN11KO,ES,Bone,Ewing Sarcoma,Ewing Sarcoma,ES,None,CVCL_C6N6,...,"CRISPR/Cas9 HGNC 26633, SLFN11",None,None,None,None,None,EWS-FLI,NaN,None,NaN
ACH-003580,PT-0hCGTm,SJ-DIPGX7,SJDIPGX7,DIPG,CNS/Brain,Diffuse Glioma,Diffuse Intrinsic Pontine Glioma,DIPG,None,None,...,None,None,None,None,None,None,None,NaN,None,NaN
ACH-003587,PT-g8kcwe,531MII,531MII,OS,Bone,Osteosarcoma,Osteosarcoma,OS,None,None,...,None,None,None,None,None,None,None,NaN,None,NaN


In [18]:
for lineage in Model_filtered['OncotreeLineage'].unique():
    print (lineage)

Ovary/Fallopian Tube
Myeloid
Bowel
Bladder/Urinary Tract
Lung
Skin
Breast
Pancreas
Lymphoid
CNS/Brain
Soft Tissue
Bone
Peripheral Nervous System
Pleura
Prostate
Kidney
Fibroblast
Biliary Tract
Esophagus/Stomach
Head and Neck
Thyroid
Ampulla of Vater
Uterus
Liver
Cervix
Eye
Vulva/Vagina
Adrenal Gland
Testis
None
Other


In [38]:
grouped_model = pd.DataFrame()

grouped_model = Model_filtered.groupby('OncotreeLineage')

print(grouped_model.groups)




{'Adrenal Gland': ['ACH-001401'], 'Ampulla of Vater': ['ACH-000182', 'ACH-000377', 'ACH-001862', 'ACH-002023', 'ACH-003338', 'ACH-003339', 'ACH-003340', 'ACH-003358'], 'Biliary Tract': ['ACH-000141', 'ACH-000209', 'ACH-000268', 'ACH-000461', 'ACH-000976', 'ACH-001494', 'ACH-001536', 'ACH-001538', 'ACH-001607', 'ACH-001619', 'ACH-001673', 'ACH-001834', 'ACH-001835', 'ACH-001836', 'ACH-001838', 'ACH-001839', 'ACH-001841', 'ACH-001842', 'ACH-001843', 'ACH-001844', 'ACH-001848', 'ACH-001849', 'ACH-001850', 'ACH-001852', 'ACH-001856', 'ACH-001857', 'ACH-001858', 'ACH-001861', 'ACH-001863', 'ACH-001864', 'ACH-001959', 'ACH-001960', 'ACH-001961', 'ACH-001997', 'ACH-002647', 'ACH-003368'], 'Bladder/Urinary Tract': ['ACH-000011', 'ACH-000018', 'ACH-000026', 'ACH-000127', 'ACH-000142', 'ACH-000242', 'ACH-000396', 'ACH-000473', 'ACH-000486', 'ACH-000522', 'ACH-000545', 'ACH-000547', 'ACH-000593', 'ACH-000720', 'ACH-000724', 'ACH-000741', 'ACH-000753', 'ACH-000802', 'ACH-000834', 'ACH-000839', 'AC

In [38]:
Model_filtered.groupby('OncotreeLineage')

In [ ]:
Model_filtered

,PatientID,CellLineName,StrippedCellLineName,DepmapModelType,OncotreeLineage,OncotreePrimaryDisease,OncotreeSubtype,OncotreeCode,PatientSubtypeFeatures,RRID,...,EngineeredModelDetails,CulturedResistanceDrug,PublicComments,CCLEName,HCMIID,ModelAvailableInDbgap,ModelSubtypeFeatures,WTSIMasterCellID,SangerModelID,COSMICID
ModelID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,PT-gj46wT,NIH:OVCAR-3,NIHOVCAR3,HGSOC,Ovary/Fallopian Tube,Ovarian Epithelial Tumor,High-Grade Serous Ovarian Cancer,HGSOC,None,CVCL_0465,...,None,None,None,NIHOVCAR3_OVARY,None,Available in SRA,None,2201.0,SIDM00105,905933.0
ACH-000004,PT-q4K2cp,HEL,HEL,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,"JAK2 and TP53 mutations,",CVCL_0001,...,None,None,None,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,None,Available in SRA,None,783.0,SIDM00594,907053.0
ACH-000005,PT-q4K2cp,HEL 92.1.7,HEL9217,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,JAK2 and TP53 mutations,CVCL_2481,...,None,None,None,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,None,Available in SRA,None,NaN,SIDM00593,NaN
ACH-000007,PT-NOXwpH,LS513,LS513,COAD,Bowel,Colorectal Adenocarcinoma,Colon Adenocarcinoma,COAD,None,CVCL_1386,...,None,None,None,LS513_LARGE_INTESTINE,None,Available in SRA,None,569.0,SIDM00677,907795.0
ACH-000009,PT-puKIyc,C2BBe1,C2BBE1,COAD,Bowel,Colorectal Adenocarcinoma,Colon Adenocarcinoma,COAD,None,CVCL_1096,...,None,None,None,C2BBE1_LARGE_INTESTINE,None,Available in SRA,None,2104.0,SIDM01233,910700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-003578,PT-XwatVo,ES8_SLFN11-KO,ES8SLFN11KO,ES,Bone,Ewing Sarcoma,Ewing Sarcoma,ES,None,CVCL_C6N6,...,"CRISPR/Cas9 HGNC 26633, SLFN11",None,None,None,None,None,EWS-FLI,NaN,None,NaN
ACH-003580,PT-0hCGTm,SJ-DIPGX7,SJDIPGX7,DIPG,CNS/Brain,Diffuse Glioma,Diffuse Intrinsic Pontine Glioma,DIPG,None,None,...,None,None,None,None,None,None,None,NaN,None,NaN
ACH-003587,PT-g8kcwe,531MII,531MII,OS,Bone,Osteosarcoma,Osteosarcoma,OS,None,None,...,None,None,None,None,None,None,None,NaN,None,NaN


In [ ]:

correlation_df_list = []
for lineage, group in Model_filtered.groupby('OncotreeLineage'):
    model_ids = group.index.values
   
    if len(group.index) < 2:
        continue
    
    E3_expr_filtered_lineage = E3_expr_filtered.loc[model_ids,:]
    crispr_dependencies_3_filtered_lineage = crispr_dependencies_3_filtered.loc[model_ids,:]
    pvals = []
    corrs = []
    tfs = []
    e3s = []
    
    for e3_col in E3_expr_filtered_lineage.columns:
        for tf_col in crispr_dependencies_3_filtered_lineage.columns:
        # Compute Pearson correlation and p-value
            corr, p_value = pearsonr(E3_expr_filtered_lineage[e3_col], crispr_dependencies_3_filtered_lineage[tf_col])
        # Append results to the DataFrame
            pvals.append(p_value)
            corrs.append(corr)
            tfs.append(tf_col)
            e3s.append(e3_col)

    correlation_df = pd.DataFrame({
        "lineage" : lineage,
        "TF" : tfs,
        "E3_ligase" : e3s,
        "p-val": pvals,
        "corr": corrs
    })
    correlation_df_list.append(correlation_df)

/var/folders/6x/6w9bj9wn3cl8pj2yyxgs7m500000gn/T/ipykernel_72302/2595111494.py:18: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, p_value = pearsonr(E3_expr_filtered_lineage[e3_col], crispr_dependencies_3_filtered_lineage[tf_col])
